# Job Setup and Post-Processing for Nath & Thingbaijam (2012)

Mostly minor reformatting.

## Job Setup

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import glob
import codecs
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText

In [ ]:
# these imports have been obsolete since the introduction of --export csv
# associated code needs re-writing to work
import openquake.oq_output.hazard_curve_converter as hcc
import openquake.oq_output.hazard_map_converter as hmc

In [ ]:
sys.path.append('../utilities')
import source_model_tools as smt  # noqa
import toolbox as tb  # noqa
import gmpe_tools as gmt

In [ ]:
nath_path = '../Data/nath2012probabilistic'
sites_table = os.path.join(nath_path, 'Table 3.csv')
df_table3 = pd.read_csv(sites_table, skiprows=1)
df_table3

In [ ]:
df_sites = df_table3[['Longitude (°E)', 'Latitude (°N)']]
sites_csv = 'NT2012_Table_3_lon_lat.csv'
print('sites_csv = %s' % sites_csv)
df_sites.to_csv(sites_csv, header=False, index=False, float_format='%g')

In [ ]:
map_imts = ['PGA','SA(0.2)','SA(1.0)']
map_tables = ['India_pga.csv','India_psa_pt2sec.csv','India_psa_1sec.csv']
map_table_list = [os.path.join(nath_path, item) for item in map_tables]
map_table_df_list = [pd.read_csv(file_name) for file_name in map_table_list]

In [ ]:
df_map = map_table_df_list[0][['lon', 'lat']]
map_csv = 'NT2012_Figure_7_Indian_subcontinent_lon_lat.csv'
print('sites_csv = %s' % map_csv)
df_map.to_csv(map_csv, header=False, index=False, float_format='%g')

In [ ]:
reduced_spacing = 2.
himalaya_csv = 'NT2012_Figure_5_Himalayas_lon_lat.csv'
df_Himalayas = pd.read_csv(himalaya_csv, names=['lon','lat'])
keep = (((df_Himalayas['lon'] % reduced_spacing) == 0) &
        ((df_Himalayas['lat'] % reduced_spacing) == 0))
reduced_csv = himalaya_csv.replace('.csv', '_reduced.csv')
df_Himalayas[keep].to_csv(reduced_csv, index=False, header=False)

In [ ]:
# areal_only_no_fmd_uncertainty_map
n_gmpe_lt_rlzs = 3*3*2*2*3*2*3*3*2
print('n_gmpe_lt_rlzs = %d' % n_gmpe_lt_rlzs)
n_src_lt_rlzs = 1
print('n_src_lt_rlzs = %d' % n_src_lt_rlzs)
n_lt_rlzs = n_gmpe_lt_rlzs*n_src_lt_rlzs
print('n_lt_rlzs = %d' % n_lt_rlzs)
n_sites = len(df_map)
print('n_sites = %d' % n_sites)
n_imt_iml = len(map_imts)
print('n_imt_iml = %d' % n_imt_iml)
n_curves = n_lt_rlzs*n_sites*n_imt_iml
print('n_curves = n_lt_rlzs*n_sites*n_imt_iml = %d*%d*%d = %d' %
     (n_lt_rlzs, n_sites, n_imt_iml, n_curves))
n_files = n_lt_rlzs*n_imt_iml
print('n_files = n_lt_rlzs*n_imt_iml = %d*%d = %d' %
     (n_lt_rlzs, n_imt_iml, n_files))

## Post-Processing

In [ ]:
CALC_ID = 123
export_dir = '../HazardOutputs'
export_path = os.path.join(os.path.abspath(export_dir), 'calc_%d' % CALC_ID)

exported_files = [os.path.join(dir_path, f)
    for dir_path, dirnames, files in os.walk(export_path)
    for f in files if f.endswith('.xml')]
exported_files
print(export_path)

In [ ]:
subset_keys = ['hazard_curve-', 'mean']
output_subset = [item for item in exported_files if all(key in item for key in subset_keys)]
output_subset

In [ ]:
plot_cities = np.array([['Bangalore', 'Bhuj', 'Chennai'],
                        ['Guwahati', 'Jabalpur', 'Kolkata'],
                        ['Koyna', 'Mumbai', 'New Delhi']])
a_maxima = np.array([[1, 2.5, 1], [2, 1.2, 1.2], [2, 1.2, 1.2]])

fig, axes = plt.subplots(plot_cities.shape[0], plot_cities.shape[1], 
                         figsize=(12,8))

for file_name in output_subset:
    hcm = hcc.read_hazard_curves(file_name)
    label = os.path.split(file_name)[1]
    
    if hcm['investigation_time'] != T_inv:
        warnings.warn(
            'Investigation time %g instead of expected %g' %
            hcm['investigation_time'], T_inv, UserWarning)
    
    if hcm['imt'] == 'PGV':
        warnings.warn('IMT %s units not properly reflected in axis label.' % 
                      hcm['imt'], UserWarning)
        
    for row in hcm['curves']:
        lon, lat = row[:2]
        in_table3 = ((df_table3['Longitude (°E)'] == lon) & 
                     (df_table3['Latitude (°N)'] == lat))
        
        if any(in_table3):
            city = df_table3.loc[in_table3, 'City'].values[0]
        else:
            print('Unknown site %g°N %g°E. Ignoring ...' % (lat, lon))
            continue
            
        if city in plot_cities:
            ax = axes[plot_cities == city][0]
            poes = row[2:]
            ax.semilogy(hcm["imls"], poes, label=hcm['imt'])

for ax, city, a_max in zip(axes.ravel(), plot_cities.ravel(), a_maxima.ravel()):
    ax.set_xlim((0, a_max))
    ax.set_ylim((1e-4, ax.get_ylim()[1]))
    for prob, label, linestyle in zip(poes_inv, poe_labels, ['--', '-.', ':']):
        ax.axhline(prob, label=label, color='0.5', linestyle=linestyle)
    anchored_text = AnchoredText(city, loc=tb.LOC_CODE['upper center'], frameon=False)
    ax.add_artist(anchored_text)

axes[1,0].set_ylabel('Annual probability of exceedence')
axes[-1,1].set_xlabel('Ground Acceleration [g]')
axes[1,-1].legend(loc='center left', frameon=False, bbox_to_anchor=(1, 0.5))
fig.savefig('Figure_6_Calc_%d.pdf' % CALC_ID, dpi=600)

In [ ]:
subset_keys = ['hazard_map', 'mean']
output_subset = [item for item in exported_files if all(key in item for key in subset_keys)]
output_subset

In [ ]:
map_csv_list = [hmc.save_hazard_map_to_csv(file_name, force_overwrite=True) for file_name in output_subset]

In [ ]:
map_csv_list

In [ ]:
df_pga = pd.read_csv(map_csv_list[0], header=1)

df_compare = pd.merge(df_table3, df_pga, left_on = 'Longitude (°E)', right_on = 'lon')
drop_cols = ['BIS','zone','GSHAP','Other','Reference','lon','lat']
df_compare.drop(drop_cols, axis=1, inplace=True)
df_compare.rename(columns={'Present':'NT2012', 'iml': 'A2016'}, inplace=True)
df_compare['Error (%)'] = (100*(df_compare['A2016']/df_compare['NT2012'] - 1)).round(1)
df_compare.sort(columns='NT2012')

In [ ]:
df_compare['Error (%)'].describe()

In [ ]:
with codecs.open('Table_3_Calc_%d.tex' % CALC_ID, 'w', 'utf-8') as file_object:
    df_compare.to_latex(file_object)

In [ ]:
df_compare.to_csv('Table_3_Calc_%d.csv' % CALC_ID)

In [ ]:
import math
math.log10(3**223)